**https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/Basics/custom_dataset**

In [ ]:
!pip install tensorflow tensorboard tf-keras-vis grad-cam
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
from PIL import Image
import matplotlib.pyplot as plt

class Emphysema_Dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        image = Image.fromarray(image).convert('RGB')  # Convert the NumPy array to a PIL Image
        y_label = torch.tensor(self.annotations.iloc[index, 1])

        if self.transform:
            image = self.transform(image)

        return image, y_label

%load_ext tensorboard

In [ ]:
#%tensorboard --logdir logs/resnet

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from pathlib import Path
from PIL import Image
#import tmm

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='./logs/resnet')

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device,"||",torch.device)

# Load Emphysema_Dataset  (host domain)
transform = transforms.Compose([
  transforms.Resize((224, 224)),  # Resize images to the size expected by ResNet50 originally 1024 x1024
  transforms.ToTensor(),
  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # Normalize based on ImageNet stats
])


dataset = Emphysema_Dataset(csv_file = Path("/kaggle/input/emphysema-dataset-from-nih-chest-x-ray/Data_Entry_2024.csv"), root_dir= Path("/kaggle/input/emphysema-dataset-from-nih-chest-x-ray/emphReform/emphReform"), transform=transform)

train_size = int(0.3 * len(dataset)) ## 0.6 is 60% split for train and testclea
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

In [5]:
def modelResnet():

    model = models.resnet50(pretrained=True)


    #model = timm.create_model('xception', pretrained=True)
    #model.eval()

    model.fc = nn.Linear(model.fc.in_features, 2)  # Emphysema_Dataset has 2 classes
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)


    return model, criterion, optimizer

In [6]:
from torchvision.models import vit_b_16

def modelViT():

    model = vit_b_16(pretrained=True)

    #model.fc = nn.Linear(model.fc.in_features, 2)  # Emphysema_Dataset has 2 classes
    model.heads.head = nn.Linear(model.heads.head.in_features, 2)
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(model)


    return model, criterion, optimizer

In [7]:

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    print("i am here with", num_epochs)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Log the training loss
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        writer.add_scalar('Training Loss', epoch_loss, epoch + 1)


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


    # Log the test accuracy
    writer.add_scalar('Test Accuracy', accuracy, epoch + 1)

In [8]:
# Train and evaluate the model
#model = Image.fromarray(model)

WhichModel = input("what model, resnet or vit? ")
if WhichModel == "resnet":
    model, criterion, optimizer = modelResnet()
elif WhichModel == "vit":
    model, criterion, optimizer = modelViT()
else:
    raise Exception("Sorry, Chose from list")

epochs = input("How Many Epochs?: ")



what model, resnet or vit? resnet


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

  warnings.warn(

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.

  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth

100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s]


How Many Epochs?: 1


In [10]:
train_model(model, train_loader, criterion, optimizer, num_epochs=int(epochs))

i am here with 1

Epoch 1/1

----------

Epoch [1/1], Loss: 0.7109

Accuracy of the model on the test images: 57.68%


https://github.com/jacobgil/pytorch-grad-cam

In [15]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, deprocess_image, preprocess_image
import cv2
import numpy as np


rgb_img = cv2.imread("/kaggle/input/emphysema-dataset-from-nih-chest-x-ray/emphReform/emphReform/00000002_000.png", 1)[:, :, ::-1]
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]).to(device)

target_layers = [model.layer4[-1]]
# Note: input_tensor can be a batch tensor with several images!

# We have to specify the target we want to generate the CAM for.
targets = [ClassifierOutputTarget(1)]

# Construct the CAM object once, and then re-use it on many images.
with GradCAM(model=model, target_layers=target_layers) as cam:
  # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
  grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
  # In this example grayscale_cam has only one image in the batch:
  grayscale_cam = grayscale_cam[0, :]
  visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
  # You can also get the model outputs without having to redo inference
  model_outputs = cam.outputs

  import matplotlib.pyplot as plt
  plt.imshow(visualization)
  plt.axis('off')
  plt.show()

NameError: name 'inverse_cams' is not defined

In [ ]:
writer.close()
# Save the trained model

In [ ]:
torch.save(model.state_dict(), 'trained_model.pth')


In [ ]:
%tensorboard --logdir logs/resnet